# ML for Hydrology - Model Proposals

In [ ]:
%pip install scikit-learn
%pip install pandas

In [23]:
# Proposal 1: Use a simple linear regression model
import pandas as pd
from sklearn import linear_model

df = pd.read_csv('./PlatteRiverDataset.csv')
stage = df['Stage'].values.reshape(-1, 1)
surge = df['Discharge'].values.reshape(-1, 1)
generic_features = df[['grayMean','graySigma','entropyMean','entropySigma','hMean','hSigma','sMean','sSigma','vMean','vSigma']].values
print(stage.shape, surge.shape, generic_features.shape)

# Train the model
model = linear_model.LinearRegression()
test_stage = stage[0:20000]
test_features = generic_features[0:20000]
model.fit(test_features, test_stage)

# Test the model
test_features = generic_features[20000:40000]
print(model.predict(X=test_features))

(42059, 1) (42059, 1) (42059, 10)
[[3.23228715]
 [3.28284917]
 [3.37234473]
 ...
 [3.64448562]
 [3.75102873]
 [3.79966697]]
